<a href="https://colab.research.google.com/github/Heity94/WalkieLookie/blob/main/TWSM_Lab/Project/Notebooks/TE_fine_tuning_SBert_SynDet_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Fine tune the sentence transformer on the synonyms datasets (for synonym detection)

In [2]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.1 MB/s 
     |████████████████████████████████| 4.4 MB 39.0 MB/s 
     |████████████████████████████████| 1.2 MB 51.9 MB/s 
     |████████████████████████████████| 101 kB 6.8 MB/s 
     |████████████████████████████████| 596 kB 6.5 MB/s 
     |████████████████████████████████| 6.6 MB 30.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=00e8ab5edca048db0f95643668cb313b3ccebcc3f2d0975ca1a73427ba119d9e
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
#Mount drive
from google.colab import drive
drive.mount('/content/drive')
data_path = "/content/drive/MyDrive/Colab Notebooks/TWSM Project/2022_Analytics Lab Student Projects/Data/Topic 1/Data_Team1/SynDet/Synonym_Dataset/"

Mounted at /content/drive


In [4]:
import pandas as pd

In [5]:
#load dfs
df_train = pd.read_csv(data_path+"SynDet_ontology_train_syn.csv", index_col=0)
df_val = pd.read_csv(data_path+"SynDet_ontology_val_syn.csv", index_col=0)
df_test = pd.read_csv(data_path+"SynDet_ontology_test_syn.csv", index_col=0)

In [6]:
df_train.head(5)
# is synonym = 0 == False

,entity_id,category,label,synonym,is_synonym
235036,individual participant,study object,PARTICIPANTS,usable and information agent,1
189726,individual participant,study object,PARTICIPANTS,professional and senior research executives,1
291037,individual participant,study object,PARTICIPANTS,business and IS IS informant,1
31468,survey,research method,COLLECTION_METHOD,telephone based postal questionnaire descriptions,1
288690,individual participant,study object,PARTICIPANTS,business and professional individual,1


In [7]:
#import packages
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, BinaryClassificationEvaluator
import logging
from datetime import datetime
import os
import gzip
import csv

In [8]:
# Load the model
model_name = 'nli-distilroberta-base-v2'

In [9]:
# Load pre-trained sentence transformer model
model = SentenceTransformer(model_name) 

Downloading:   0%|          | 0.00/736 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.71k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [10]:
train_samples = []
for index, row in df_train.iterrows():
  inp_example = InputExample(texts=[row['entity_id'], row['synonym']], label=float(row['is_synonym']))
  train_samples.append(inp_example)

In [11]:
len(train_samples)

567429

In [12]:
evaluation_samples = []
for index, row in df_val.iterrows():
  inp_example = InputExample(texts=[row['entity_id'], row['synonym']], label=float(row['is_synonym']))
  evaluation_samples.append(inp_example)

In [13]:
test_samples = []
for index, row in df_test.iterrows():
  inp_example = InputExample(texts=[row['entity_id'], row['synonym']], label=float(row['is_synonym']))
  test_samples.append(inp_example)

In [14]:
# An evaluator evaluates the model performance during training on held-out dev data. It is used to determine the best model that is saved to disc.
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(evaluation_samples)

## As the baseline for sentence transformer is 84%, we improve the fine tuning by choosing different parameters

### Model 1

In [15]:
#train a new model
model_data_path_1 = "/content/drive/MyDrive/Colab Notebooks/TWSM Project/2022_Analytics Lab Student Projects/Data/Topic 1/Data_Team1/SynDet/Synonym_Dataset/model_sentence_transformer_result_1"

In [ ]:
#Definition of train dataset, dataloader and train loss
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

In [ ]:
#Fine tune model
model.fit(train_objectives=[(train_dataloader, train_loss)], 
          epochs=2, #Number of epochs for training
          warmup_steps=100, #WarmupLinear (default), the learning rate is increased from o up to the maximal learning rate
          evaluator=evaluator, 
          evaluation_steps=1000, # If > 0, evaluate the model using evaluator after each number of training steps
          output_path=model_data_path_1)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/35465 [00:00<?, ?it/s]

In [16]:
model_1 = SentenceTransformer(model_data_path_1)
test_evaluator = BinaryClassificationEvaluator.from_input_examples(test_samples)
test_evaluator(model_1, output_path=model_data_path_1)

0.9979238234289713

- Model 1 result: 99,79 %

### Model 2

Increasing the batch size from 16 to 32, the number of epochs for training from 2 to 4 and the evaluation steps to 2000.

In [ ]:
#train a new model
model_data_path_2 = "/content/drive/MyDrive/Colab Notebooks/TWSM Project/2022_Analytics Lab Student Projects/Data/Topic 1/Data_Team1/SynDet/Synonym_Dataset/model_sentence_transformer_result_2"

In [ ]:
#Definition of train dataset, dataloader and train loss
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=32)
train_loss = losses.CosineSimilarityLoss(model)

In [ ]:
num_epochs = 4
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up

In [ ]:
#Fine tune model
model.fit(train_objectives=[(train_dataloader, train_loss)], 
          epochs=num_epochs, #Number of epochs for training
          warmup_steps=warmup_steps, #WarmupLinear (default), the learning rate is increased from o up to the maximal learning rate
          evaluator=evaluator, 
          evaluation_steps=2000, # If > 0, evaluate the model using evaluator after each number of training steps
          output_path=model_data_path_2)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/17733 [00:00<?, ?it/s]

Iteration:   0%|          | 0/17733 [00:00<?, ?it/s]

Iteration:   0%|          | 0/17733 [00:00<?, ?it/s]

Iteration:   0%|          | 0/17733 [00:00<?, ?it/s]

In [ ]:
model_2 = SentenceTransformer(model_data_path_2)
test_evaluator = BinaryClassificationEvaluator.from_input_examples(test_samples)
test_evaluator(model_2, output_path=model_data_path_2)

0.9992936840396178

- Model 2 result: 99.92 %

### Model 3

Increasing the batch size to 128 and the evaluation steps to 5000

In [ ]:
#train a new model
model_data_path_3 = "/content/drive/MyDrive/Colab Notebooks/TWSM Project/2022_Analytics Lab Student Projects/Data/Topic 1/Data_Team1/SynDet/Synonym_Dataset/model_sentence_transformer_result_3"

In [ ]:
#Definition of train dataset, dataloader and train loss
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=128)
train_loss = losses.CosineSimilarityLoss(model)

In [ ]:
num_epochs = 4
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up

In [ ]:
#Fine tune model
model.fit(train_objectives=[(train_dataloader, train_loss)], 
          epochs = num_epochs, #Number of epochs for training
          warmup_steps=warmup_steps, #WarmupLinear (default), the learning rate is increased from o up to the maximal learning rate
          evaluator=evaluator, 
          evaluation_steps=5000,
          output_path=model_data_path_3)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4434 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4434 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4434 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4434 [00:00<?, ?it/s]

In [ ]:

model_3 = SentenceTransformer(model_data_path_3)
test_evaluator = BinaryClassificationEvaluator.from_input_examples(test_samples)
test_evaluator(model_3, output_path=model_data_path_3)

0.9990611706468856

- Model 3 result: 99.9 %